In [1]:
%reload_ext autoreload
%autoreload 2
# %matplotlib qt

from time import time
from tqdm import tqdm
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

from cellector.utils import get_s2p_data, get_s2p_redcell
from cellector.roi_processor import RoiProcessor
from cellector.gui import SelectionGUI

In [2]:
# Gather suite2p data
s2p_dir = Path(r"C:\Users\Andrew\Documents\localData\ATL027\2023-07-28\701\suite2p")
stats, references = get_s2p_data(s2p_dir)
red_s2p = get_s2p_redcell(s2p_dir)
extra_features = {"red_s2p": red_s2p}

# Create a redcell processing object
rcp = RoiProcessor(stats, references, extra_features=extra_features)

# Next step: 
# 1. Implement a saving method (and maybe a two way system for creating a red cell object ? from suite2p or from scratch)
# 2. Visualization methods for optimizing the hyperparameters for preprocessing
# 3. Implement an optimization method for phase_corr_eps? 
#    - Might just be as simple as the user inputing the maximum size ROI, and then measuring
#      the average power for higher frequencies...

In [3]:
gui = SelectionGUI(rcp)

In [27]:
class Pipeline:
    def __init__(self, name, method, dependencies):
        self.name = name
        self.method = method
        self.dependencies = dependencies

pipeline = Pipeline("name", lambda self: "Andii", [])

class TestClass:
    """A class that collects and computes features using decorated methods."""
    
    def __init__(self):
        self.features = {}
        self._feature_pipelines = []
    
    def register_pipeline(self, pipeline):
        if not isinstance(pipeline, Pipeline):
            raise ValueError("Pipeline must be an instance of Pipeline.")
        self._feature_pipelines.append(pipeline)
    
    def compute_features(self):
        """Compute all registered features and store results."""
        for pipeline in self._feature_pipelines:
            self.features[pipeline.name] = pipeline.method(self)

# Usage
test = TestClass()
test.register_pipeline(pipeline)
test.compute_features()
print(test.features)  # {'name': 'Andii', 'age': 21}

{'name': 'Andii'}
